In [1]:
import geemap, ee

ee.Initialize()

roi = geemap.shp_to_ee(r'D:/work/new/shp/fw.shp')
fw = roi.geometry()
# Map.addLayer(fw,{},'fw')
sample = geemap.shp_to_ee(r'D:\work\new\point\new.shp')
Map = geemap.Map()
# Map

In [2]:
def EVI(img):
    nir = img.select("SR_B4")
    red = img.select("SR_B3")
    blue = img.select("SR_B1")
    evi = img.expression("2.5*(B4 - B3)/(B4 + 6*B3 - 7.5*B1 + 1)",{"B4": nir,"B3": red ,"B1": blue})
    evi = evi.rename('EVI')
    return evi

def NDVI(img):
    ndvi = img.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
    return ndvi

In [3]:
# CRU  https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
# bands = ['pr','tem','SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','LSWI','NDVI','EVI','GCVI','RVI','EVI2','SAVI','slope']
bands = ['pr','tem','NDVI','EVI','slope','elevation','SR_B3','SR_B4']
label = 'CID'

# 计算平均气温
def fun(image):
    return (image.select('tmmx').add(image.select('tmmn'))).multiply(0.5).rename('tem')

# 投影
def project(image):
    return image.reproject(ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").first().select(0).projection())

# L457去云
def maskL457sr(image):
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Unused
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(31).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
      .addBands(thermalBand, None, True) \
      .updateMask(qaMask) \
      .updateMask(saturationMask).copyProperties(image, ["system:index","system:time_start"])

def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111',2)).eq(0);
    saturationMask = image.select('QA_RADSAT').eq(0);

#   // Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
    
    return image.addBands(opticalBands, None, True)\
                 .addBands(thermalBands, None, True)\
                 .updateMask(qaMask)\
                 .updateMask(saturationMask)\
                 .copyProperties(image, ["system:time_start",'system:id'])

# 通过年份获取数据
def handle(year):
    image = []
    precipitation = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                    .filterBounds(fw)\
                    .filter(ee.Filter.calendarRange(year,year,'year'))\
                    .select('pr')\
                    .sum().clip(fw)
    image.append(project(precipitation))

    temperature = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                    .filterBounds(fw)\
                    .filter(ee.Filter.calendarRange(year,year,'year'))\
                    .select(['tmmn','tmmx'])\
                    .map(fun).mean().multiply(0.1)\
                    .clip(fw)
    image.append(project(temperature))
    
    dataset = ee.Image('USGS/SRTMGL1_003').clip(fw)
    elevation = dataset.select('elevation')
    slope = ee.Terrain.slope(elevation).rename('slope')
    image.append(elevation)
    image.append(slope)
    
    image = ee.Image(image)
    
    if year==1980:
        l5m = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
                    .filterBounds(fw)\
                    .filter(ee.Filter.calendarRange(year,year+6,'year'))\
                    .filterMetadata('CLOUD_COVER', 'less_than', 25)\
                    .sort('CLOUD_COVER', False)\
                    .map(maskL457sr)\
                    .mean()
        return image.addBands(l5m)
        
    elif year<=2010:
        l5m = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
            .filterBounds(fw)\
            .filter(ee.Filter.calendarRange(year-5,year+5,'year'))\
            .filterMetadata('CLOUD_COVER', 'less_than', 25)\
            .sort('CLOUD_COVER', False)\
            .map(maskL457sr)\
            .mean()
        return image.addBands(l5m)
    else:
        l8m = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
                    .filterBounds(fw)\
                    .filter(ee.Filter.calendarRange(year-5,year+5,'year'))\
                    .filterMetadata('CLOUD_COVER', 'less_than', 25)\
                    .sort('CLOUD_COVER', False)\
                    .map(maskL8sr)\
                    .mean()
        return image.addBands(l8m)
    
def RandomForest(tree):
    classifier = ee.Classifier.smileRandomForest(tree).train(trainingPartition,label,bands)
    return testingPartition.classify(classifier).errorMatrix(label, 'classification').accuracy()

In [4]:
# # 'LSWI','NDVI','EVI','GCVI','RVI','EVI2','SAVI'
# image_1980 = handle(1980)
# image_1980 = image_1980.addBands(NDVI(image_1980))
# image_1980 = image_1980.addBands(EVI(image_1980))

# image_2020 = handle(2020)
# image_2020 = image_2020.addBands(NDVI(image_2020))
# image_2020 = image_2020.addBands(EVI(image_2020))

image2 = handle(2020)
image2 = image2.addBands(NDVI(image2))
image2 = image2.addBands(EVI(image2))

In [5]:
from GEE_xiong import ee_export
import os
outpath = r'C:\Users\xiong\Downloads\out'

for year in range(1985,2020,5):
    
    year = 2020
    
    outfile = outpath+os.sep+'{}'.format(year)
    if os.path.exists(outfile+'.tif'):
        continue
        
    print('{}_'.format(year))
#     image1 = handle(year)

#     image1 = image1.addBands(NDVI(image1))
#     image1 = image1.addBands(EVI(image1))

    # 特征值的提取
    training = image2.select(bands).sampleRegions(**{
        'collection':sample,
        'scale':30,
        'geometries': True
    })
        
    #  random uniforms to the training dataset.
    withRandom = training.randomColumn() #样本点随机的排列

    # 我们想保留一些数据进行测试，以避免模型过度拟合
    split = 0.8

    trainingPartition = withRandom.filter(ee.Filter.lt('random', split))#筛选80%的样本作为训练样本
    testingPartition = withRandom.filter(ee.Filter.gte('random', split))#筛选20%的样本作为测试样本

    # 分类方法选择smileCart() randomForest() minimumDistance libsvm
    classifier = ee.Classifier.smileRandomForest(100).train(trainingPartition,label,bands)
#     classified = image1.select(bands).classify(classifier)
#     classified = classified.where(image2.select('NDVI').lte(0),0)

    # 运用测试样本分类，确定要进行函数运算的数据集以及函数
    test = testingPartition.classify(classifier)
    test_accuracy = test.errorMatrix(label, 'classification')

    print('总体精度:',test_accuracy.accuracy().getInfo())
#         print('{}精度:'.format(i),test_accuracy.accuracy().getInfo())

#     ee_export(fw, classified, outfile, 1000)
    break

2020_


In [6]:
numTrees = ee.List.sequence(80, 160, 5)
accuracies = numTrees.map(RandomForest)

In [ ]:
accuracies.getInfo()